## GIS-Based Landfill Site Analysis: Criteria Evaluation and Optimization Data Preparation

In [ ]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, box
import rasterio
import rasterio.errors
from rasterio.warp import transform
import pyogrio.errors 
import osmnx as ox
import numpy as np
from scipy.ndimage import sobel 
import time 

# --- Helper Functions (Optimized for loop) ---

def get_nearest_osm_dist(lat, lon, tags, max_dist=30000, point_crs="EPSG:4326", target_calculation_crs="EPSG:32626"):
    """Queries OSM for the nearest geometry to (lat, lon) matching the provided tags."""
    try:
        pt_geom = Point(lon, lat)
        pt_gdf = gpd.GeoDataFrame([{'id': 1, 'geometry': pt_geom}], crs=point_crs)
        ox.settings.use_cache = True
        ox.settings.log_console = False 
        gdf_osm = ox.features_from_point(center_point=(lat, lon), tags=tags, dist=max_dist)
        if gdf_osm.empty: return float('nan')
        pt_proj = pt_gdf.to_crs(target_calculation_crs)
        gdf_osm_proj = gdf_osm.to_crs(target_calculation_crs)
        if gdf_osm_proj.empty: return float('nan')
        distances = gdf_osm_proj.geometry.distance(pt_proj.geometry.iloc[0])
        return distances.min() if not distances.empty else float('nan')
    except Exception:
        return float('nan')

def get_raster_value_from_file(lat, lon, raster_path):
    """Reads a value from a single-band raster (like pre-calculated slope) at the specified point."""
    try:
        with rasterio.open(raster_path) as src:
            xs, ys = transform("EPSG:4326", src.crs, [lon], [lat])
            x_proj, y_proj = xs[0], ys[0]
            if not (src.bounds.left <= x_proj <= src.bounds.right and src.bounds.bottom <= y_proj <= src.bounds.top):
                return float('nan')
            row, col = src.index(x_proj, y_proj)
            if not (0 <= row < src.height and 0 <= col < src.width): return float('nan')
            value_array = src.read(1, window=rasterio.windows.Window(col, row, 1, 1))
            if value_array is None or value_array.size == 0: return float('nan')
            value = float(value_array[0, 0])
            nodata = src.nodata
            if nodata is not None and abs(value - nodata) < 1e-6: return float('nan')
            return value
    except Exception:
        return float('nan')

def get_aspect_at_point(lat, lon, dem_path_with_elevation_band):
    """
    Calculates terrain aspect in degrees for a point (lat, lon)
    from band 1 (elevation) of a DEM file.
    Returns aspect (0-360 degrees, 0=North, -1 for flat areas) or float('nan').
    """
    try:
        with rasterio.open(dem_path_with_elevation_band) as src:
            raster_crs = src.crs
            bounds = src.bounds

            xs, ys = transform("EPSG:4326", raster_crs, [lon], [lat])
            x_proj, y_proj = xs[0], ys[0]

            xmin, ymin, xmax, ymax = bounds
            if not (xmin <= x_proj <= xmax and ymin <= y_proj <= ymax):
                print(f"Projected point ({x_proj:.2f}, {y_proj:.2f}) is outside DEM bounds.")
                return float('nan')

            dem_array = src.read(1).astype(np.float32)
            nodata_value = src.nodata

            cell_size_x = src.transform.a
            cell_size_y = abs(src.transform.e)

            if nodata_value is not None:
                dem_proc = np.where(dem_array == nodata_value, np.nan, dem_array)
            else:
                dem_proc = dem_array

            dz_dx = sobel(dem_proc, axis=1, mode='reflect') / (8 * cell_size_x)
            dz_dy_raw = sobel(dem_proc, axis=0, mode='reflect') / (8 * cell_size_y)

            aspect_rad = np.arctan2(dz_dx, -dz_dy_raw)
            aspect_deg_cartesian = np.degrees(aspect_rad)

            aspect_final = (90.0 - aspect_deg_cartesian + 360.0) % 360.0

            flat_mask = (np.abs(dz_dx) < 1e-9) & (np.abs(dz_dy_raw) < 1e-9)
            aspect_final[flat_mask] = -1.0

            if nodata_value is not None:
                aspect_final = np.where(np.isnan(dem_proc), nodata_value, aspect_final)

            row, col = src.index(x_proj, y_proj)
            if not (0 <= row < aspect_final.shape[0] and 0 <= col < aspect_final.shape[1]):
                print(f"Pixel indices ({row}, {col}) are outside aspect array bounds.")
                return float('nan')

            aspect_at_point = aspect_final[row, col]
            if nodata_value is not None and abs(aspect_at_point - nodata_value) < 1e-6:
                print(f"Aspect value at ({lat}, {lon}) is NoData.")
                return float('nan')

            if abs(aspect_at_point - (-1.0)) < 1e-6:
                print(f"Area at ({lat}, {lon}) is flat (aspect = -1).")
            else:
                print(f"Terrain aspect at ({lat}, {lon}): {aspect_at_point:.2f} degrees (0=North).")

            return aspect_at_point

    except rasterio.errors.RasterioIOError as e:
        print(f"I/O error reading DEM file '{dem_path_with_elevation_band}': {e}.")
        return float('nan')
    except Exception as e:
        print(f"Unexpected error calculating aspect at ({lat}, {lon}): {e}")
        return float('nan')

def get_distance_to_feature(lat, lon, feature_gdf, point_crs="EPSG:4326"):
    """Calculates distance from a point to a pre-loaded and reprojected GeoDataFrame of features."""
    try:
        if feature_gdf is None or feature_gdf.empty: return float('nan')
        pt_geom = Point(lon, lat)
        pt_gdf = gpd.GeoDataFrame([{'id': 1, 'geometry': pt_geom}], crs=point_crs).to_crs(feature_gdf.crs)
        pt_point = pt_gdf.geometry.iloc[0]
        distances = feature_gdf.geometry.distance(pt_point)
        return distances.min()
    except Exception:
        return float('nan')

# --- MAIN WORKFLOW START ---

# 1. GENERATE CANDIDATE POINTS
# ========================================
print("Step 1: Generating candidate areas and points...")
island_boundary_path = ".././data/fogo_geology/fogo_geology.shp"
grid_spacing_meters = 1000
utm_crs = "EPSG:32626"
candidate_points_latlon = None

if not os.path.exists(island_boundary_path):
    print(f"ERROR: Island boundary file '{island_boundary_path}' not found.")
else:
    try:
        island_boundary = gpd.read_file(island_boundary_path)
        island_boundary_utm = island_boundary.to_crs(utm_crs)
        dissolved_boundary = island_boundary_utm.dissolve()
        xmin, ymin, xmax, ymax = dissolved_boundary.total_bounds
        
        grid_x = np.arange(np.floor(xmin), np.ceil(xmax), grid_spacing_meters)
        grid_y = np.arange(np.floor(ymin), np.ceil(ymax), grid_spacing_meters)
        polygons = [box(x, y, x + grid_spacing_meters, y + grid_spacing_meters) for x in grid_x for y in grid_y]
        grid_gdf = gpd.GeoDataFrame(geometry=polygons, crs=utm_crs)
        
        final_grid = gpd.sjoin(grid_gdf, dissolved_boundary, how="inner", predicate="intersects")
        final_grid = final_grid.drop_duplicates(subset=['geometry']).drop(columns=['index_right'], errors='ignore')

        centroid_points_utm = final_grid.copy()
        centroid_points_utm['geometry'] = centroid_points_utm.geometry.centroid
        
        candidate_points_latlon = centroid_points_utm.to_crs("EPSG:4326")
        
        print(f"Total of {len(candidate_points_latlon)} candidate points generated.")
        print("The variable 'candidate_points_latlon' is ready for analysis.")

    except Exception as e:
        print(f"An error occurred during point generation: {e}")

# 2. ANALYZE CRITERIA FOR EACH CANDIDATE POINT
# ======================================================
if candidate_points_latlon is not None and not candidate_points_latlon.empty:
    print("\nStep 2: Starting criteria analysis for each candidate point...")
    
    # --- Data file path definitions ---
    dem_elevation_utm_path = '.././data/tif/slope_gdal_degrees.tif'
    dem_slope_gdal_path = '.././data/tif/slope_gdal_degrees.tif'
    geology_shp_path = '.././data/geology_with_permeability/geology_with_permeability.shp'
    protected_areas_shp_path = '.././data/protected_area/protected_area.shp'

    # --- Pre-loading vector data for optimization ---
    print("Pre-loading vector files...")
    metric_crs = "EPSG:32626"
    low_perm_gdf, protected_areas_gdf = None, None
    try:
        geology_gdf = gpd.read_file(geology_shp_path)
        if geology_gdf.crs is None: geology_gdf.set_crs("EPSG:4326", inplace=True)
        low_perm_gdf = geology_gdf[geology_gdf['permeability'] == 'Low'].to_crs(metric_crs)
        
        protected_areas_gdf = gpd.read_file(protected_areas_shp_path)
        if protected_areas_gdf.crs is None: protected_areas_gdf.set_crs("EPSG:4326", inplace=True)
        protected_areas_gdf = protected_areas_gdf.to_crs(metric_crs)
    except Exception as e:
        print(f"Error pre-loading vector files: {e}.")

    # --- Criteria to be analyzed ---
    osm_criteria = {
        'distance_to_urban_areas': {'tags': {'landuse': ['residential', 'commercial', 'retail', 'industrial']}},
        'distance_to_aerodrome':   {'tags': {'aeroway': 'aerodrome'}},
        'distance_to_rivers':      {'tags': {'waterway': ['stream', 'river', 'canal']}},
        'distance_to_roads':       {'tags': {'highway': ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified', 'residential', 'service', 'road', 'track', 'path']}},
        'distance_to_water_infrastructure': {'tags': {'man_made': ['water_well', 'borehole', 'petroleum_well', 'water_tap', 'water_works']}},
        'distance_to_cultivated_areas': {'tags': {'landuse': ['farmland', 'farmyard', 'orchard', 'vineyard', 'allotments']}},
        'distance_to_coastline':    {'tags': {'natural': 'coastline'}},
    }

    # --- Analysis Loop ---
    real_data = []
    start_time = time.time()
    
    total_points = len(candidate_points_latlon)
    for i, (index, point) in enumerate(candidate_points_latlon.iterrows()):
        lat = point.geometry.y
        lon = point.geometry.x
        
        print(f"--- Analyzing Point {i + 1}/{total_points}: Lat={lat:.4f}, Lon={lon:.4f} ---")
        
        point_results = {'latitude (degrees)': lat, 'longitude (degrees)': lon}

        # --- Data Collection and Processing ---
        # Raster
        slope = get_raster_value_from_file(lat, lon, dem_slope_gdal_path)
        point_results['slope (degrees)'] = round(slope, 4) if pd.notna(slope) else np.nan
        
        aspect = get_aspect_at_point(lat, lon, dem_elevation_utm_path)
        point_results['aspect (degrees)'] = round(aspect, 4) if pd.notna(aspect) else np.nan

        # OSM
        for criteria_name, criteria_info in osm_criteria.items():
            osm_dist = get_nearest_osm_dist(lat, lon, criteria_info['tags'])
            point_results[criteria_name + ' (m)'] = round(osm_dist, 4) if pd.notna(osm_dist) else np.nan
        
        # Distance to Vectors
        dist_perm = get_distance_to_feature(lat, lon, low_perm_gdf)
        point_results['distance_to_low_permeability_zones (m)'] = round(dist_perm, 4) if pd.notna(dist_perm) else np.nan
        
        dist_protected = get_distance_to_feature(lat, lon, protected_areas_gdf)
        point_results['distance_to_protected_areas (m)'] = round(dist_protected, 4) if pd.notna(dist_protected) else np.nan
        
        real_data.append(point_results)

    duration = time.time() - start_time

    # --- Finalization ---
    results_df = pd.DataFrame(real_data)
    
    print("\n--- Analysis completed for all points. ---")
    print(f"Total analysis time for {total_points} points: {duration:.2f} seconds.")
    
    # --- MODIFICATION: Rename Columns to Include Units ---
    print("\nStep 3: Renaming columns in final DataFrame...")
    column_rename_map = {
        'latitude (degrees)': 'latitude (degrees)',
        'longitude (degrees)': 'longitude (degrees)',
        'slope (degrees)': 'slope (degrees)',
        'aspect (degrees)': 'aspect (degrees)',
        'distance_to_urban_areas (m)': 'distance_to_urban_areas (m)',
        'distance_to_aerodrome (m)': 'distance_to_aerodrome (m)',
        'distance_to_rivers (m)': 'distance_to_rivers (m)',
        'distance_to_roads (m)': 'distance_to_roads (m)',
        'distance_to_water_infrastructure (m)': 'distance_to_water_infrastructure (m)',
        'distance_to_cultivated_areas (m)': 'distance_to_cultivated_areas (m)',
        'distance_to_coastline (m)': 'distance_to_coastline (m)',
        'distance_to_low_permeability_zones (m)': 'distance_to_low_permeability_zones (m)',
        'distance_to_protected_areas (m)': 'distance_to_protected_areas (m)'
    }
    results_df.rename(columns=column_rename_map, inplace=True)
    
    print("\nSample of final DataFrame with renamed columns:")
    print(results_df.head())

    output_csv_path = "criteria_data_for_optimization_524-3.csv"
    results_df.to_csv(output_csv_path, index=False)
    print(f"\nComplete DataFrame saved to: '{output_csv_path}'")
else:
    print("\nNo candidate points were generated. Analysis cannot proceed.")

---
## Criteria Data Standardization

In [ ]:
import pandas as pd
import os

def standardize_csv(file_path):
    # Read the original CSV
    df = pd.read_csv(file_path)

    # Identify column types
    coord_columns = ['latitude (degrees)', 'longitude (degrees)']
    degree_columns = ['slope (degrees)', 'aspect (degrees)']
    distance_columns = [col for col in df.columns if '(m)' in col]

    # Create standardized DataFrame
    formatted_df = df.copy()

    # Apply specific formatting
    formatted_df[coord_columns] = df[coord_columns].round(4)
    formatted_df[degree_columns] = df[degree_columns].round(2)
    formatted_df[distance_columns] = df[distance_columns].round(0).astype(int)

    # Generate new filename
    base, ext = os.path.splitext(file_path)
    new_file = f"{base}_standardized{ext}"

    # Save to CSV
    formatted_df.to_csv(new_file, index=False)

    print(f"Standardized file saved as: {new_file}")

# Example usage
standardize_csv("criteria_data_for_optimization_524-3.csv")

---
## Pareto Optimization: Cost vs Environmental Impact vs Safety Trade-offs

In [ ]:
import numpy as np
import pandas as pd
from pymoo.core.problem import ElementwiseProblem
from pymoo.core.sampling import Sampling
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.visualization.scatter import Scatter
import os

# --- 1. Load Real Data ---
data_path = "criteria_data_for_optimization_524-3_standardized-organized.csv"

if not os.path.exists(data_path):
    print(f"ERROR: Data file '{data_path}' not found.")
else:
    try:
        df = pd.read_csv(data_path, delimiter=';')
    except Exception:
        df = pd.read_csv(data_path)
        
    print(f"Data loaded successfully. Total of {len(df)} candidate areas for optimization.")
    
    # --- Column Name Adjustments ---
    # Rename CSV columns to match what the optimization script expects
    # The permeability criterion is now distance-based
    name_mapping = {
        'latitude (degrees)': 'latitude',
        'longitude (degrees)': 'longitude',
        'slope (degrees)': 'Slope (degrees)',
        'aspect (degrees)': 'Aspect (degrees)',
        'distance_to_urban_areas (m)': 'Urban Areas (m)',
        'distance_to_aerodrome (m)': 'Aerodrome (m)',
        'distance_to_rivers (m)': 'Rivers (m)',
        'distance_to_roads (m)': 'Road Network (m)',
        'distance_to_water_infrastructure (m)': 'Wells/Boreholes (m)',
        'distance_to_cultivated_areas (m)': 'Cultivated Areas (m)',
        'distance_to_coastline (m)': 'Coastline (m)',
        'distance_to_low_permeability_zones (m)': 'Permeability (Distance) (m)', # UPDATED
        'distance_to_protected_areas (m)': 'Protected Areas (m)'
    }
    df.rename(columns=name_mapping, inplace=True)
    
    # AHP Weights (Andrade and Barbosa, 2015, Table 5)
    # Key "Permeability (mD)" changed to "Permeability (Distance) (m)"
    ahp_weights = {
        "Urban Areas (m)": 0.166, "Aerodrome (m)": 0.102, 
        "Permeability (Distance) (m)": 0.152, # UPDATED
        "Protected Areas (m)": 0.066, "Coastline (m)": 0.069, "Rivers (m)": 0.091,
        "Slope (degrees)": 0.113, "Road Network (m)": 0.030, "Aspect (degrees)": 0.075,
        "Wells/Boreholes (m)": 0.055, "Cultivated Areas (m)": 0.081
    }

    # Min and Max for normalization - CALCULATED FROM REAL DATA
    valid_columns = [col for col in ahp_weights.keys() if col in df.columns]
    min_max_real = {col: (df[col].min(), df[col].max()) for col in valid_columns}
    print("\nLimits (Min, Max) calculated from real data for normalization:")
    print(min_max_real)

    # 2. Define objective functions for each area
    def cost_function(row):
        total_cost_score = 0
        # Permeability (Distance): Greater distance to low permeability areas -> Higher cost (worse)
        val_p, min_p, max_p = row["Permeability (Distance) (m)"], min_max_real["Permeability (Distance) (m)"][0], min_max_real["Permeability (Distance) (m)"][1]
        norm_p = (val_p - min_p) / (max_p - min_p) if (max_p - min_p) > 0 else 0
        total_cost_score += ahp_weights["Permeability (Distance) (m)"] * norm_p
        
        # Slope: Higher slope -> Higher cost (worse)
        val_d, min_d, max_d = row["Slope (degrees)"], min_max_real["Slope (degrees)"][0], min_max_real["Slope (degrees)"][1]
        norm_d = (val_d - min_d) / (max_d - min_d) if (max_d - min_d) > 0 else 0
        total_cost_score += ahp_weights["Slope (degrees)"] * norm_d
        
        # Road Network: Greater distance -> Higher cost (worse)
        val_rv, min_rv, max_rv = row["Road Network (m)"], min_max_real["Road Network (m)"][0], min_max_real["Road Network (m)"][1]
        norm_rv = (val_rv - min_rv) / (max_rv - min_rv) if (max_rv - min_rv) > 0 else 0
        total_cost_score += ahp_weights["Road Network (m)"] * norm_rv
        
        return total_cost_score

    def environmental_impact_function(row):
        total_impact_score = 0
        # Distance Criteria: Smaller distance -> Higher impact (worse)
        for criterion in ["Protected Areas (m)", "Coastline (m)", "Rivers (m)", "Wells/Boreholes (m)"]:
            val, min_val, max_val = row[criterion], min_max_real[criterion][0], min_max_real[criterion][1]
            norm_val = (max_val - val) / (max_val - min_val) if (max_val - min_val) > 0 else 0
            total_impact_score += ahp_weights[criterion] * norm_val
        
        # Aspect: Outside 90-180 range is worse
        val_ev = row["Aspect (degrees)"]
        min_ev, max_ev = 90, 180 # Favorable range
        norm_ev = 0 if (val_ev >= min_ev and val_ev <= max_ev) else 1
        total_impact_score += ahp_weights["Aspect (degrees)"] * norm_ev
        
        return total_impact_score

    def safety_function(row):
        total_safety_score = 0
        # Distance Criteria: Greater distance -> Higher safety (better)
        for criterion in ["Urban Areas (m)", "Aerodrome (m)", "Cultivated Areas (m)"]:
             val, min_val, max_val = row[criterion], min_max_real[criterion][0], min_max_real[criterion][1]
             norm_val = (val - min_val) / (max_val - min_val) if (max_val - min_val) > 0 else 0
             total_safety_score += ahp_weights[criterion] * norm_val
        return total_safety_score

    df["CostDef"] = df.apply(cost_function, axis=1)
    df["ImpactDef"] = df.apply(environmental_impact_function, axis=1)
    df["SafetyDef"] = df.apply(safety_function, axis=1)

    class MySampling(Sampling):
        def _do(self, problem, n_samples, **kwargs):
            X = np.full((n_samples, problem.n_var), False, dtype=bool)
            for i in range(n_samples):
                idx = np.random.choice(problem.n_var, problem.n_select, replace=False)
                X[i, idx] = True
            return X

    class SelectAreas(ElementwiseProblem):
        def __init__(self, df, n_select=3):
            super().__init__(n_var=len(df), n_obj=3, xl=0, xu=1, type_var=bool)
            self.df = df
            self.n_select = n_select
        def _evaluate(self, x, out, *args, **kwargs):
            if x.sum() != self.n_select:
                out["F"] = [1e10, 1e10, 1e10]
                return
            idx = np.where(x)[0]
            cost = self.df.iloc[idx]["CostDef"].sum()
            impact = self.df.iloc[idx]["ImpactDef"].sum()
            positive_safety = self.df.iloc[idx]["SafetyDef"].sum()
            out["F"] = [cost, impact, -positive_safety]

    problem = SelectAreas(df, n_select=3)
    algorithm = NSGA2(pop_size=100, sampling=MySampling())
    res = minimize(problem, algorithm, ('n_gen', 100), seed=1, verbose=True)

    # 4. Show best solutions (Pareto frontier)
    if res.F is not None and len(res.F) > 0:
        pareto = res.F
        valid_solutions_mask = (pareto[:, 0] < 1e9)
        pareto_valid = pareto[valid_solutions_mask]
        if len(pareto_valid) > 0:
            print("\nOptimal solutions (Cost, Impact, -Safety):")
            print(pareto_valid)
            plot = Scatter(title="Pareto Frontier", labels=["Cost", "Impact", "-Safety"])
            plot.add(pareto_valid)
            plot.show()

            if res.X is not None:
                X_valid = res.X[valid_solutions_mask]
                if len(X_valid) > 0:
                    # Lowest cost solution
                    idx_lowest_cost = np.argmin(pareto_valid[:,0])
                    selected_cost = np.where(X_valid[idx_lowest_cost])[0]
                    print("\nAreas selected in LOWEST COST solution:")
                    print(df.iloc[selected_cost])
                    
                    # Lowest environmental impact solution
                    idx_lowest_impact = np.argmin(pareto_valid[:,1])
                    selected_impact = np.where(X_valid[idx_lowest_impact])[0]
                    print("\nAreas selected in LOWEST ENVIRONMENTAL IMPACT solution:")
                    print(df.iloc[selected_impact])
                    
                    # HIGHEST SAFETY solution (lowest value of -Safety)
                    idx_highest_safety = np.argmin(pareto_valid[:,2])
                    selected_safety = np.where(X_valid[idx_highest_safety])[0]
                    print("\nAreas selected in HIGHEST SAFETY solution:")
                    print(df.iloc[selected_safety])
    else:
        print("\nNo solutions found by the algorithm.")

---
## Multi-Objective Optimization Results: Visualizing the Pareto Frontier

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Extract valid Pareto solutions
pareto = res.F
valid_solutions_mask = (pareto[:, 0] < 1e9)
pareto_valid = pareto[valid_solutions_mask]
n_solutions = len(pareto_valid)

if n_solutions > 0:
    # Set consistent styling
    plt.style.use('seaborn-v0_8-bright')
    
    # 1. 3D Pareto Front Visualization
    fig_3d = plt.figure(figsize=(12, 9))
    ax_3d = fig_3d.add_subplot(111, projection='3d')
    
    # Create color gradient based on solution index
    colors = plt.cm.viridis(np.linspace(0, 1, n_solutions))
    
    scatter_3d = ax_3d.scatter(
        pareto_valid[:, 0],  # Cost
        pareto_valid[:, 1],  # Environmental Impact
        pareto_valid[:, 2],  # -Safety
        c=colors, s=60, alpha=0.8, depthshade=True
    )
    
    # Labels and title
    ax_3d.set_xlabel('\nCost (minimize)', fontsize=12, linespacing=2)
    ax_3d.set_ylabel('\nEnvironmental Impact (minimize)', fontsize=12, linespacing=2)
    ax_3d.set_zlabel('\n-Safety (maximize)', fontsize=12, linespacing=2)
    ax_3d.set_title(
        f'Pareto Front: Trade-off distribution among Cost, Impact, and Safety objectives\nacross {n_solutions} non-dominated solutions',
        fontsize=14, pad=20
    )
    
    # Grid and viewing angle
    ax_3d.grid(True, linestyle='--', alpha=0.4)
    ax_3d.view_init(elev=25, azim=45)
    
    # 2. 2D Pairwise Visualizations
    fig_2d, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 6))
    fig_2d.suptitle(
        f'Pairwise Trade-off Analysis of {n_solutions} Pareto-optimal Solutions',
        fontsize=16, y=1.02
    )
    
    # Cost vs Environmental Impact
    ax1.scatter(pareto_valid[:, 0], pareto_valid[:, 1], c=colors, alpha=0.8)
    ax1.set_xlabel('Cost (minimize)', fontsize=12)
    ax1.set_ylabel('Environmental Impact (minimize)', fontsize=12)
    ax1.set_title('Cost vs Environmental Impact', fontsize=13)
    ax1.grid(True, linestyle='--', alpha=0.4)
    
    # Cost vs Safety
    ax2.scatter(pareto_valid[:, 0], pareto_valid[:, 2], c=colors, alpha=0.8)
    ax2.set_xlabel('Cost (minimize)', fontsize=12)
    ax2.set_ylabel('-Safety (maximize)', fontsize=12)
    ax2.set_title('Cost vs Safety', fontsize=13)
    ax2.grid(True, linestyle='--', alpha=0.4)
    
    # Environmental Impact vs Safety
    ax3.scatter(pareto_valid[:, 1], pareto_valid[:, 2], c=colors, alpha=0.8)
    ax3.set_xlabel('Environmental Impact (minimize)', fontsize=12)
    ax3.set_ylabel('-Safety (maximize)', fontsize=12)
    ax3.set_title('Environmental Impact vs Safety', fontsize=13)
    ax3.grid(True, linestyle='--', alpha=0.4)
    
    plt.tight_layout()
    plt.show()

else:
    print("No valid Pareto solutions to plot.")

---
## Spatial Criteria Interdependencies: Spearman Correlation Analysis

In [ ]:
import pandas as pd
import numpy as np

# File path
file_path = 'criteria_data_for_optimization_524-3_standardized-organized.csv'

# EXACT column names in the CSV file, as per your diagnosis
actual_column_names = [
    'slope (degrees)', 'aspect (degrees)', 'distance_to_urban_areas (m)',
    'distance_to_aerodrome (m)', 'distance_to_rivers (m)', 'distance_to_roads (m)',
    'distance_to_water_infrastructure (m)', 'distance_to_cultivated_areas (m)', 'distance_to_coastline (m)',
    'distance_to_low_permeability_zones (m)', 'distance_to_protected_areas (m)'
]

# "Friendly" names for final presentation in tables and text.
# The order here MUST match the order in actual_column_names
friendly_column_names = {
    'slope (degrees)': 'Slope (degrees)',
    'aspect (degrees)': 'Aspect (degrees)',
    'distance_to_urban_areas (m)': 'Dist. Urban Areas (m)',
    'distance_to_aerodrome (m)': 'Dist. Aerodrome (m)',
    'distance_to_rivers (m)': 'Dist. Rivers (m)',
    'distance_to_roads (m)': 'Dist. Road Network (m)',
    'distance_to_water_infrastructure (m)': 'Dist. Wells/Boreholes (m)',
    'distance_to_cultivated_areas (m)': 'Dist. Cultivated Areas (m)',
    'distance_to_coastline (m)': 'Dist. Coastline (m)',
    'distance_to_low_permeability_zones (m)': 'Dist. Low Permeability Zones (m)',
    'distance_to_protected_areas (m)': 'Dist. Protected Areas (m)'
}

# Load DataFrame with correct delimiter
df = pd.read_csv(file_path, sep=';')

# Calculate Spearman correlation matrix using ACTUAL column names
corr_matrix = df[actual_column_names].corr(method='spearman')

# Rename matrix indices and columns to "friendly" names for presentation
corr_matrix.rename(columns=friendly_column_names, index=friendly_column_names, inplace=True)

# Transform matrix into pairs list
corr_pairs = corr_matrix.unstack().reset_index()
corr_pairs.columns = ['Criterion 1', 'Criterion 2', 'Spearman_rho']
corr_pairs = corr_pairs[corr_pairs['Criterion 1'] != corr_pairs['Criterion 2']]
corr_pairs['sorted_criteria'] = corr_pairs.apply(lambda r: tuple(sorted((r['Criterion 1'], r['Criterion 2']))), axis=1)
corr_pairs = corr_pairs.drop_duplicates(subset='sorted_criteria').drop(columns='sorted_criteria')
corr_pairs['abs_rho'] = corr_pairs['Spearman_rho'].abs()
strongest_pairs = corr_pairs.sort_values(by='abs_rho', ascending=False).drop(columns='abs_rho')

# Final Results
print(">>> CRITERIA PAIRS WITH STRONGEST CORRELATION (Spearman's ρ) <<<")
print("Actual values calculated from your data:")
print(strongest_pairs.head(5).round(2).to_string(index=False))